<a href="https://colab.research.google.com/github/ksdkamesh99/LowLightEnhancer/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
cd /content/drive/My Drive/LowLightEnhancement

/content/drive/My Drive/LowLightEnhancement


In [54]:
import tensorflow as tf
import numpy as np
from PIL import Image

In [55]:
img_high=np.load("image_high.npy")

In [56]:
img_low=np.load("image_low.npy")

In [57]:
img_high.shape

(1485, 150, 150, 3)

## Illumination Mask Network

In [58]:
top_inp=tf.keras.layers.Input(shape=(150,150,3))
top=tf.keras.layers.Conv2D(64,kernel_size=(3,3),input_shape=(150,150,3),activation='relu')(top_inp)
top=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(top)

In [59]:
bottom_inp=tf.keras.layers.Input(shape=(150,150,3))
bottom_resize=tf.keras.layers.Lambda( 
    lambda image: tf.compat.v1.image.resize_nearest_neighbor( 
        image, 
        (224, 224)
    )
)(bottom_inp)


In [60]:
bottom=tf.keras.layers.Conv2D(64,kernel_size=(3,3),input_shape=(224,224,3),activation='relu')(bottom_resize)
bottom=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(bottom)
bottom=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(bottom)
bottom=tf.keras.layers.Lambda( 
    lambda image: tf.compat.v1.image.resize_nearest_neighbor( 
        image, 
        (146,146)
    )
)(bottom)

In [61]:
bottom.get_shape()

TensorShape([None, 146, 146, 64])

In [62]:
top.get_shape()

TensorShape([None, 146, 146, 64])

In [63]:
merged=tf.keras.layers.concatenate([top,bottom],axis=-1)

In [64]:
merged=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(merged)
merged=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(merged)
merged=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(merged)

In [65]:
merged.get_shape()

TensorShape([None, 140, 140, 64])

In [66]:
model_illumination_mask=tf.keras.models.Model(inputs=[top_inp,bottom_inp],outputs=merged)

In [67]:
model_illumination_mask.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 224, 224, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_22 (Conv2D)              (None, 222, 222, 64) 1792        lambda_8[0][0]                   
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 150, 150, 3) 0                                            
_______________________________________________________________________________________

# Illumination Map 

In [68]:
merged=tf.keras.layers.Lambda( 
    lambda image: tf.compat.v1.image.resize_nearest_neighbor( 
        image, 
        (150,150)
    )
)(merged)

In [69]:
merged.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'lambda_10/ResizeNearestNeighbor:0' shape=(None, 150, 150, 64) dtype=float32>>

In [70]:
ieb1=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(merged)
ieb1=tf.keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu')(ieb1)

In [71]:
ieb1.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'conv2d_29/Relu:0' shape=(None, 146, 146, 64) dtype=float32>>

In [72]:
max_pool=tf.keras.layers.GlobalMaxPooling2D()(ieb1)

In [73]:
avg_pool=tf.keras.layers.GlobalAveragePooling2D()(ieb1)

In [74]:
max_pool.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'global_max_pooling2d_2/Max:0' shape=(None, 64) dtype=float32>>

In [75]:
dense1=tf.keras.layers.Dense(8,activation='relu',kernel_initializer='he_normal')
dense2=tf.keras.layers.Dense(64,activation='sigmoid',kernel_initializer='he_normal')
  

In [76]:
max_pool=dense1(max_pool)
max_pool=dense2(max_pool)

avg_pool=dense1(avg_pool)
avg_pool=dense2(avg_pool)

In [77]:
max_pool

<tf.Tensor 'dense_7/Sigmoid:0' shape=(None, 64) dtype=float32>

In [78]:
max_pool=tf.keras.layers.Lambda(
    lambda image: tf.expand_dims(tf.expand_dims(image,axis=1),axis=1)
)(max_pool)
avg_pool=tf.keras.layers.Lambda(
    lambda image: tf.expand_dims(tf.expand_dims(image,axis=1),axis=1)
)(avg_pool)

In [79]:
max_pool.get_shape

<bound method Tensor.get_shape of <tf.Tensor 'lambda_11/ExpandDims_1:0' shape=(None, 1, 1, 64) dtype=float32>>

In [81]:
channel=tf.keras.layers.Add()([max_pool,avg_pool])

In [82]:
channel

<tf.Tensor 'add_1/add:0' shape=(None, 1, 1, 64) dtype=float32>

In [84]:
ieb1=tf.keras.layers.Multiply()([ieb1,channel])

In [85]:
ieb1

<tf.Tensor 'multiply_1/mul:0' shape=(None, 146, 146, 64) dtype=float32>